In [1]:
"""
Galois field multiplication
"""
def gf_mult_noLUT(x: int, y: int, prim: int = 0) -> int:
    """
    Multiplication in Galois Fields without using a precomputed look-up table.
    It performs carry-less multiplication and then, if prim > 0, reduces modulo the
    prime polynomial `prim`.

    :param x: First operand (as an integer representing a GF(2) polynomial)
    :param y: Second operand
    :param prim: Primitive polynomial for modular reduction (default 0 means no reduction)
    :return: The product in GF(2^n) (with modular reduction if prim > 0)
    """

    def cl_mult(x: int, y: int) -> int:
        """Carry-less multiplication of two integers (treating them as GF(2) polynomials)."""
        z: int = 0
        i: int = 0
        while (y >> i) > 0:
            if y & (1 << i):
                z ^= x << i
            i += 1
        return z

    def bit_length(n: int) -> int:
        """
        Compute the position of the most significant 1-bit in an integer.
        Equivalent to int.bit_length() built-in function.
        """
        bits: int = 0
        while n >> bits:
            bits += 1
        return bits

    def cl_div(dividend: int, divisor: int) -> int:
        """
        Carry-less long division of two integers.
        Returns the remainder of dividend/divisor (using XOR as subtraction).
        Equivalent to polynomial remainder of polynomial division.
        """
        dl1: int = bit_length(dividend)
        dl2: int = bit_length(divisor)
        if dl1 < dl2:
            return dividend
        # Align the most significant bit of divisor with that of dividend, 
        # so remainder will be at most 1 bit less than divisor (9-bit divisor, 8-bit or less remainder)
        for i in range(dl1 - dl2, -1, -1):
            if dividend & (1 << (i + dl2 - 1)):
                dividend ^= divisor << i
        return dividend

    # Perform carry-less multiplication
    result: int = cl_mult(x, y)

    # If a primitive polynomial is provided, reduce the product modulo prim
    if prim > 0:
        result = cl_div(result, prim)

    return result

In [3]:
"""
The following code is for testing that non-primitve polynomials don't generate all the elements.
There are duplicates and, therefore, missing elements.
"""
def test_multiplicative_cycle(prim, generator):
    """
    Starting from 1, repeatedly multiply by the candidate generator using GF multiplication.
    Return the list of generated nonzero field elements.
    For a field defined by a primitive polynomial, if 'generator' is primitive then
    this should yield a cycle of 255 unique elements.
    """
    cycle = []
    current = 1
    for _ in range(255):  # There are 255 nonzero elements in GF(2^8)
        cycle.append(current)
        current = gf_mult_noLUT(current, generator, prim)
    return cycle


def find_best_generator(prim):
    """
    Try candidate generators from 2 to 255 (skipping 0 and 1) and return the one with the
    largest multiplicative cycle length.
    """
    best_gen = None
    best_cycle_len = 0
    for gen in range(2, 256):
        cycle = test_multiplicative_cycle(prim, gen)
        cycle_len = len(set(cycle))
        if cycle_len > best_cycle_len:
            best_cycle_len = cycle_len
            best_gen = gen
    return best_gen, best_cycle_len


# Set up two tests:
# 1. Using a known primitive polynomial for GF(2^8) (for example, 0x11d).
# 2. Using a non-primitive polynomial (for example, 300).

primitive_poly = 0x11d  # Example: 0x11d (285 in decimal) is a known primitive polynomial for GF(2^8)
nonprimitive_poly = 257 # An example non-primitive polynomial

# Test with the primitive polynomial.
best_gen_prim, cycle_len_prim = find_best_generator(primitive_poly)
cycle_prim = test_multiplicative_cycle(primitive_poly, best_gen_prim)

print("Testing with primitive polynomial (0x11d):")
print("  Best candidate generator:", best_gen_prim)
print("  Cycle length (unique elements):", cycle_len_prim)
if cycle_len_prim == 255:
    print("  --> All 255 nonzero field elements are generated. This is expected.")
else:
    print("  --> Unexpected cycle length!")

# Test with the non-primitive polynomial.
best_gen_nonprim, cycle_len_nonprim = find_best_generator(nonprimitive_poly)
cycle_nonprim = test_multiplicative_cycle(nonprimitive_poly, best_gen_nonprim)

print("\nTesting with non-primitive polynomial (300):")
print("  Best candidate generator:", best_gen_nonprim)
print("  Cycle length (unique elements):", cycle_len_nonprim)
if cycle_len_nonprim == 255:
    print("  --> (Unexpected) All 255 nonzero field elements are generated!")
else:
    print(f"  --> The cycle is shorter, indicating duplicates/missing elements.")


# Optional: To see some of the cycle values for visual inspection:
print("\nCycle with primitive polynomial:")
print(cycle_prim)
print(f"\nCycle with non-primitive polynomial (first {cycle_len_nonprim+1} elements, first duplicate is {cycle_nonprim[cycle_len_nonprim]}):")
print(cycle_nonprim[:(cycle_len_nonprim+1)])

Testing with primitive polynomial (0x11d):
  Best candidate generator: 2
  Cycle length (unique elements): 255
  --> All 255 nonzero field elements are generated. This is expected.

Testing with non-primitive polynomial (300):
  Best candidate generator: 3
  Cycle length (unique elements): 9
  --> The cycle is shorter, indicating duplicates/missing elements.

Cycle with primitive polynomial:
[1, 2, 4, 8, 16, 32, 64, 128, 29, 58, 116, 232, 205, 135, 19, 38, 76, 152, 45, 90, 180, 117, 234, 201, 143, 3, 6, 12, 24, 48, 96, 192, 157, 39, 78, 156, 37, 74, 148, 53, 106, 212, 181, 119, 238, 193, 159, 35, 70, 140, 5, 10, 20, 40, 80, 160, 93, 186, 105, 210, 185, 111, 222, 161, 95, 190, 97, 194, 153, 47, 94, 188, 101, 202, 137, 15, 30, 60, 120, 240, 253, 231, 211, 187, 107, 214, 177, 127, 254, 225, 223, 163, 91, 182, 113, 226, 217, 175, 67, 134, 17, 34, 68, 136, 13, 26, 52, 104, 208, 189, 103, 206, 129, 31, 62, 124, 248, 237, 199, 147, 59, 118, 236, 197, 151, 51, 102, 204, 133, 23, 46, 92, 184, 1

In [23]:
def rwh_primes1(n):
    # http://stackoverflow.com/questions/2068372/fastest-way-to-list-all-primes-below-n-in-python/3035188#3035188
    # rwh for Rober William Hanks
    ''' Returns  a list of primes < n '''
    sieve = [True] * (n/2)
    for i in range(3,int(n**0.5)+1,2):
        if sieve[i/2]:
            sieve[i*i/2::i] = [False] * ((n-i*i-1)/(2*i)+1)
    return [2] + [2*i+1 for i in range(1,n/2) if sieve[i]]

def find_prime_polys(generator=2, c_exp=8, fast_primes=False, single=False):
    '''Compute the list of prime polynomials for the given generator and galois field characteristic exponent.'''
    # fast_primes will output less results but will be significantly faster.
    # single will output the first prime polynomial found, so if all you want is to just find one prime polynomial to generate the LUT for Reed-Solomon to work, then just use that.

    # A prime polynomial (necessarily irreducible) is necessary to reduce the multiplications in the Galois Field, so as to avoid overflows.
    # Why do we need a "prime polynomial"? Can't we just reduce modulo 255 (for GF(2^8) for example)? Because we need the values to be unique.
    # For example: if the generator (alpha) = 2 and c_exp = 8 (GF(2^8) == GF(256)), then the generated Galois Field (0, 1, a, a^1, a^2, ..., a^(p-1)) will be galois field it becomes 0, 1, 2, 4, 8, 16, etc. However, upon reaching 128, the next value will be doubled (ie, next power of 2), which will give 256. Then we must reduce, because we have overflowed above the maximum value of 255. But, if we modulo 255, this will generate 256 == 1. Then 2, 4, 8, 16, etc. giving us a repeating pattern of numbers. This is very bad, as it's then not anymore a bijection (ie, a non-zero value doesn't have a unique index). That's why we can't just modulo 255, but we need another number above 255, which is called the prime polynomial.
    # Why so much hassle? Because we are using precomputed look-up tables for multiplication: instead of multiplying a*b, we precompute alpha^a, alpha^b and alpha^(a+b), so that we can just use our lookup table at alpha^(a+b) and get our result. But just like in our original field we had 0,1,2,...,p-1 distinct unique values, in our "LUT" field using alpha we must have unique distinct values (we don't care that they are different from the original field as long as they are unique and distinct). That's why we need to avoid duplicated values, and to avoid duplicated values we need to use a prime irreducible polynomial.

    # Here is implemented a bruteforce approach to find all these prime polynomials, by generating every possible prime polynomials (ie, every integers between field_charac+1 and field_charac*2), and then we build the whole Galois Field, and we reject the candidate prime polynomial if it duplicates even one value or if it generates a value above field_charac (ie, cause an overflow).
    # Note that this algorithm is slow if the field is too big (above 12), because it's an exhaustive search algorithm. There are probabilistic approaches, and almost surely prime approaches, but there is no determistic polynomial time algorithm to find irreducible monic polynomials. More info can be found at: http://people.mpi-inf.mpg.de/~csaha/lectures/lec9.pdf
    # Another faster algorithm may be found at Adleman, Leonard M., and Hendrik W. Lenstra. "Finding irreducible polynomials over finite fields." Proceedings of the eighteenth annual ACM symposium on Theory of computing. ACM, 1986.

    # Prepare the finite field characteristic (2^p - 1), this also represent the maximum possible value in this field
    root_charac = 2 # we're in GF(2)
    field_charac = int(root_charac**c_exp - 1)
    field_charac_next = int(root_charac**(c_exp+1) - 1)

    prim_candidates = []
    if fast_primes:
        prim_candidates = rwh_primes1(field_charac_next) # generate maybe prime polynomials and check later if they really are irreducible
        prim_candidates = [x for x in prim_candidates if x > field_charac] # filter out too small primes
    else:
        prim_candidates = range(field_charac+2, field_charac_next, root_charac) # try each possible prime polynomial, but skip even numbers (because divisible by 2 so necessarily not irreducible)

    # Start of the main loop
    correct_primes = []
    for prim in prim_candidates: # try potential candidates primitive irreducible polys
        seen = [0] * (field_charac+1) # memory variable to indicate if a value was already generated in the field (value at index x is set to 1) or not (set to 0 by default)
        conflict = False # flag to know if there was at least one conflict

        # Second loop, build the whole Galois Field
        x = 1
        for i in range(field_charac):
            # Compute the next value in the field (ie, the next power of alpha/generator)
            x = gf_mult_noLUT(x, generator, prim, field_charac+1)

            # Rejection criterion: if the value overflowed (above field_charac) or is a duplicate of a previously generated power of alpha, then we reject this polynomial (not prime)
            if x > field_charac or seen[x] == 1:
                conflict = True
                break
            # Else we flag this value as seen (to maybe detect future duplicates), and we continue onto the next power of alpha
            else:
                seen[x] = 1

        # End of the second loop: if there's no conflict (no overflow nor duplicated value), this is a prime polynomial!
        if not conflict: 
            correct_primes.append(prim)
            if single: return prim

    # Return the list of all prime polynomials
    return correct_primes # you can use the following to print the hexadecimal representation of each prime polynomial: print [hex(i) for i in correct_primes]